#### Precipitation and Air Temperature data were downloaded from
- NASA's [Daymet](https://daymet.ornl.gov/) using coordinates from the Clemson University Pee Dee Research Center
- latitude: 34.289	
- longitude: -79.737

#### Wind and humidity data were downloaded from
- Climate [Engine](https://clim-engine.appspot.com/climateEngine) using coordinates from the Clemson Pee Dee Research and Education Center
- latitude: 34.289	
- longitude: -79.737

#### These are more precise environmental data which will be used in combination with the trait [data](https://github.com/MagicMilly/for-data-publication/tree/main/data/processed) from Brenton et al. 2016 at the Clemson University Pee Dee Research Center

In [ ]:
import datetime
import numpy as np
import pandas as pd

### A. Daymet Download

In [ ]:
daymet_0 = pd.read_csv('data/raw/clemson_raw_daily_2014_updated.csv')
print(daymet_0.shape)
# daymet_0.head()

### Columns from Daymet Download
- year
- day of year
- dayl - daylength, seconds per day
- precipitation in mm
- srad - shortwave radiation (can be used with dayl to calculate total daily radiation)
- swe - snow water equivalent
- air temperature max in C
- air temperature min in C
- vp - daily average partial pressure of water vapor, in Pascals

#### Add Dates

In [ ]:
date_range_2014 = pd.date_range(start='2014-01-01', end='2014-12-31')

In [ ]:
daymet_1 = daymet_0.copy()
daymet_1['date'] = date_range_2014
print(daymet_1.shape)
# daymet_1.tail()

#### Drop Columns
- year
- dayl
- solar radiation
- snow water equivalent

In [ ]:
cols_to_drop = ['year', 'dayl (s)', 'srad (W/m^2)', 'swe (kg/m^2)']

daymet_2 = daymet_1.drop(labels=cols_to_drop, axis=1)
print(daymet_2.shape)
# daymet_2.head()

#### Rename Columns

In [ ]:
daymet_3 = daymet_2.rename({'yday': 'year_of_day', 'prcp (mm/day)': 'precip_mm', 'tmax (deg c)': 'temp_max_c',
                           'tmin (deg c)': 'temp_min_c', 'vp (Pa)': 'vapor_pressure'}, axis=1)

print(daymet_3.shape)
# daymet_3.tail()

In [ ]:
# daymet_3.dtypes

### B. Climate Engine Download
- rh_n - dataset with relative humidity values
- wind_n - dataset with wind values
    - Assumption that these are average wind speed values per day, but have not found the official documentation yet

In [ ]:
rh_0 = pd.read_csv('data/raw/clemson_rh_2014.csv')
print(rh_0.shape)
# rh_0.head()

#### Rename Columns

In [ ]:
# rh_0.columns

In [ ]:
rh_1 = rh_0.rename({'(%) Min Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'min_rh',
                   '(%) Max Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'max_rh'}, axis=1)

print(rh_1.shape)
# rh_1.head()

#### Change `DateTime` values to datetime objects

In [ ]:
date_range = pd.to_datetime(rh_1.DateTime)
# date_range

In [ ]:
rh_2 = rh_1.copy()
rh_2['date'] = date_range
print(rh_2.dtypes)
# rh_2.head()

#### Drop DateTime Column

In [ ]:
rh_3 = rh_2.drop('DateTime', axis=1)
print(rh_3.shape)
# rh_3.tail()

In [ ]:
wind_0 = pd.read_csv('data/raw/climate_engine_wind_speed_2014.csv')
print(wind_0.shape)
# wind_0.head()

#### Rename Column

In [ ]:
wind_1 = wind_0.rename({'(m/s) Wind Speed (gridMET) at -79.7370E,34.2890N, 2014-01-01 to 2014-12-31': 'wind_speed'},
                      axis=1)

print(wind_1.shape)
# wind_1.head()

#### New Column with datetime object

In [ ]:
date_range = pd.to_datetime(wind_1.DateTime)

wind_2 = wind_1.copy()
wind_2['date'] = date_range

print(wind_2.shape)
# wind_2.tail()

#### Drop DateTime Column

In [ ]:
wind_3 = wind_2.drop('DateTime', axis=1)
# wind_3.head()

#### Merge DataFrames

In [ ]:
temp_rh_wind_0 = daymet_3.merge(rh_3, how='outer', right_on='date', left_on='date').merge(wind_3, how='outer',
                                                                                    left_on='date', right_on='date')

print(temp_rh_wind_0.shape)
temp_rh_wind_0.head()